# Breast Cancer Helper Chatbox


  1.  A chatbot which assess risk of Breast Cancer
  2.  Answers queries related to breast cancer like, treatments, cancer terms etc.


## Working of Chat Bot

Input is taken from user and is sent to Text classification model created in `Train Breast Cancer NLP Model for Chatbot`
The NLP model analysis the input from user and classifies it in one of the class from `output_df`. 
`output_df` countans the responses.

Once the classifier classifies the class, one of the response from the corresponding responses will be taken and displayed to the user.

If the content is classfied for tag `CancerRisk`, more inputs are taken from user related to medical history and that data is used to identify probable risk of future Breast Cancer using model created in `Breast Cancer Risk Predictor Model`
note book


In [1]:
import numpy as np
import pandas as pd
import json
import tensorflow as tf

In [3]:
# load responses for classes, more info on this is in `Train Breast Cancer NLP Model for Chatbot`
output_df = pd.read_json("save_replys_dataframe/output_df.json")

In [4]:
output_df.head()

,tag,responses,class
0,greetings,"[Hello, Hey!, What can I do for you?]",9
1,goodbye,"[Sad to see you go :(, Talk to you later, Good...",8
2,cancerInfo,[A disease in which abnormal cells divide unco...,5
3,BreastcancerInfo,[When cells divide uncontrollably in Breast an...,0
4,tumor,[A tumor or lump is any abnormal mass of tissu...,10


In [5]:
# import NLP model created to classify user's inputs to one of the classes mentioned in `output_df`. More info on this is providied in `Train Breast Cancer NLP Model for Chatbot`
import tensorflow as tf
import tensorflow_hub as hub
model_load  = tf.keras.models.load_model("models/NLP_model.h5", custom_objects={'KerasLayer': hub.KerasLayer})

## Check Breast Cancer risk using "Breast Cancer Risk Predictor Model"

In [6]:
# import model which predicts Breast Cancer Risk. The Model is created in `Breast Cancer Risk Predictor Model` notebook
import joblib
loadedModel = joblib.load('models/BC_risk_model.pkl')

In [7]:
def checkBreastCancerRisk():
    inputs = []
    print("Do you want to analyze your risk of getting Breast Cancer? \nyou will be asked few questions and at the end you will get a result in terms of probabilty percentage(0% means least probable and  100% means most probable)\n do you want to continue? (y/n)")
    user_input =input()
    
    if (user_input.casefold() != "yes".casefold() and   user_input.casefold() != "y".casefold()):
        print("HelperBot: No Problem, how else can I help you ?")
        return
    
    print("HelperBot: What is your Meanopause status?\n If premenopausal enter '0' if postmenopausal or if age greater than or equal to 55 enter '1', if unknown enter '9' ")
    inputs.append(int(input("Me: ")))
        
    print("HelperBot: Enter Age Group:\n '1' for 35-39; '2' for 40-44; '3' for 45-49; '4' for 50-54; '5' for 55-59; '6' for 60-64; '7' for 65-69; '8' for 70-74; '9' for 75-79; '10' for 80-84")
    inputs.append(int(input("Me: ")))
    
    print("HelperBot: Do you know your Breast Density? If you don't know please enter '9' else enter according to the below mentioned categories:\n'1' for Almost entirely fat; '2' for Scattered fibroglandular densities; '3' for Heterogeneously dense; '4' for Extremely dense; '9' for Unknown or different measurement system")
    inputs.append(int(input("Me: ")))
    
    print("HelperBot: Please mention your race as per below categories:\n'1' for white; '2' for Asian/Pacific Islander; '3' for black; '4; for Native American; '5' for other/mixed; '9' for unknown")
    inputs.append(int(input("Me: ")))
    
    print("HelperBot: Are you a Hispanic? (a person of Cuban, Mexican, Puerto Rican, South or Central American, or other Spanish culture or origin regardless of race.)\n Enter'0' for no; '1' for yes; '9' for unknown")
    inputs.append(int(input("Me: ")))
    
    print("HelperBot: Enter your bmi:\nEnter '1' if BMI falls in '10-24.99'; 2 if bmi fall in '25-29.99'; 3 if bmi falls in '30-34.99'; '4' if bmi fall in '35 or more'; '9' if unknown")
    inputs.append(int(input("Me: ")))

    print("HelperBot: What was your age when your first baby was born?\nEnter '0' if  Age was less than 30; '1' if Age was 30 or greater; Enter '2' if Nulliparous; Enter '9' if unknown")
    inputs.append(int(input("Me: ")))

    print("HelperBot: How many of your first degree relatives have or had Breast cancer?\nEnter '0' for zero; '1' for one; '2' for 2 or more; '9' for unknown`")
    inputs.append(int(input("Me: ")))
    
    print("HelperBot: Has any procedure, like biopsy or any breast related surgery was done on you?\nEnter '0' for 'no';Enter '1' = yes; '9' = unknown")
    inputs.append(int(input("Me: ")))
    
    print("HelperBot: Result of last mammogram you had:\n '0' for negative for suspected cancer; '1' for false positive for suspected cancer; '9' for unknown or none of the above. ")
    inputs.append(int(input("Me: ")))
    
    print("HelperBot: Did you have Surgical menopause:\n'0' for natural; '1' for surgical; '9' for unknown or not menopausal (menopaus=0 or menopaus=9)")
    inputs.append(int(input("Me: ")))
    
    print("HelperBot: Are you currently on hormone therapy?\nEnter '0' for no; '1' for yes; '9' for unknown or not menopausal (menopaus=0 or menopaus=9)")
    inputs.append(int(input("Me: ")))
    
    probablity = loadedModel.predict_proba([inputs])[0][1]
    
    print(f"\nYour chance of getting Breast Cancer is: {probablity*100:.2f}%")
    print("\nHelperBot: Thanks, anything else I can do for you?")

In [10]:
import random
print("HelperBot: Hi How may I help you?")
while(1):
    user_input = input("Me:")
    predicted_result = model_load.predict([user_input])
    class_output = predicted_result.argmax()
    #print(class_output)
    #print(f"The class is {output_df[output_df['class'] == class_output ]['tag']}")
    response = random.sample(output_df[output_df['class'] ==class_output]['responses'].to_list()[0],1)
    
    print(f"HelperBot: {response[0]}\n")
    
    if (output_df[output_df['class'] == class_output ]['tag'].to_list()[0] == "goodbye"):
        break
    if(output_df[output_df['class'] == class_output ]['tag'].to_list()[0] == "CancerRisk"):
        checkBreastCancerRisk()

HelperBot: Hi How may I help you?
Me:hi


InternalError: GPU sync failed